## Homework 2 Solution Template
### CSCI 4270
### Due: February 5, 2024

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import sys

In [3]:
def print_matrix(name, A):
    print(f'Matrix {name}:')
    for i in range(A.shape[0]):
        row = A[i, :]
        print(f'{row[0]:.1f}', end='')
        for j in range(1, row.shape[0]):
            print(f', {row[j]:.1f}', end='')
        print()
        
# x rotation
def Rx (xRotation):
    # calculate each row and then stack vertically
    r1 = np.array([1,0,0])
    r2 = np.array([0,np.cos(xRotation),-np.sin(xRotation)])
    r3 = np.array([0,np.sin(xRotation), np.cos(xRotation)])
    return np.vstack([r1,r2,r3])

# y rotation
def Ry (yRotation):
    # calculate each row and then stack vertically
    r1 = np.array([np.cos(yRotation),0,np.sin(yRotation)])
    r2 = np.array([0,1,0])
    r3 = np.array([-np.sin(yRotation),0,np.cos(yRotation)])
    return np.vstack([r1,r2,r3])

# z rotation
def Rz (zRotation):
    # calculate each row and then stack vertically
    r1 = np.array([np.cos(zRotation),-np.sin(zRotation),0])
    r2 = np.array([np.sin(zRotation),np.cos(zRotation),0])
    r3 = np.array([0,0,1])
    return np.vstack([r1,r2,r3])

# rotation composition
def Rotation(r):
    rx = Rx(r[0])
    ry = Ry(r[1])
    rz = Rz(r[2])
    # dot the arrays together with z as the last
    temp = np.dot(rx,ry)
    return np.dot(temp,rz)

# rotation and translation composition
def RT(r, t):
    # create an empty array and populate with the rotation and translation information
    new = np.zeros((r.shape[0],r.shape[1]+1))
    new[:,0:-1] = r
    new[:,-1] = t
    return new

# calculate K matrix
def K(p):
    f = p[0]
    # convert units (micron to mm)
    d = p[1] * 0.001 
    ic = p[2]
    jc = p[3]
    s = f/d
    # stack arrays vertically with information populated from helper functions
    row1 = np.array([s,0,jc])
    row2 = np.array([0,s,ic])
    row3 = np.array([0,0,1])
    return np.vstack([row1,row2,row3])

# form camera matrix
def M(r, t, p):
    RMatrix = Rotation(r)
    tprime = -np.dot(np.transpose(RMatrix),t)
    rt = RT(np.transpose(RMatrix), tprime)
    KMatrix = K(p)
    print_matrix("R", RMatrix)
    print_matrix("K", KMatrix)
    return np.dot(KMatrix, rt)

# determine if point is inside or outside of image plane
def decide(loc):
    row = loc[1]
    col = loc[0]
    if (0<=row<=4000 and 0<=col<=6000):
        print ('%.1f %.1f inside' % (row,col))
    else:
        print ('%.1f %.1f outside' % (row, col))

# determine if the point is above or below the plane
def visible(point,r,t):
    rmat = Rotation(r)
    # camera center
    point = rmat.dot(np.array([0,0,0])) + t
    axis = rmat.dot(np.array([0,0,1]))
    # normalized optical axis
    normalized_axis = axis/np.linalg.norm(axis)
    ret = np.inner(normalized_axis ,point-point)
    if (ret > 0): # visible
        return True
    else: # not visible
        return False

# pretty print a matrix to IO
def prettyprint(arr):
    for i in range(arr.shape[0]):
        # camera matrix is guaranteed to be 3 by 4
        outstr = "{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(arr[i][0],arr[i][1],arr[i][2],arr[i][3])
        print (outstr)

# projects a point to a plane
def project(points,r,t):
    augment = RT(points,np.array([1]*points.shape[0]))
    points_t = np.transpose(augment)
    pixle_loc = np.dot(camera_matrix,points_t)
    pixle_loc[0] = pixle_loc[0]/pixle_loc[-1]
    pixle_loc[1] = pixle_loc[1]/pixle_loc[-1]
    img_loc = pixle_loc[0:2,:]
    # printing
    vs = [] # store the indices of visible ones
    hd = [] # store the indices of the hidden ones
    print ('Projections:')
    for i in range(img_loc.shape[1]):
        outstr = '{}: {:.1f} {:.1f} {:.1f}'.format(i,points[i][0], points[i][1], points[i][2])
        print (outstr, '=> ',end = '')
        decide(img_loc[:,i])
        ret = visible(points[i], r, t)
        if ret:
            vs.append(i)
        else:
            hd.append(i)
    print ('visible:',*vs)
    print ('hidden:',*hd)

def p1_camera(parameters, points):
    r = np.array(parameters[0],dtype = np.float32)
    # convert degree to radian to use in sine and cosine calculation
    r = np.deg2rad(r)
    t = np.array(parameters[1], dtype = np.float32)
    parameters = np.array(parameters[2], dtype = np.float32)
    global camera_matrix
    camera_matrix = M(r,t,parameters)
    print_matrix("Matrix M:", camera_matrix)

    # project the points onto image
    project(np.asarray(points),r,t)

In [4]:
"""
Problem 1, Test 1. This is a very simple test that could be easily checked by hand.
"""
params = [
    [0.0, 0.0, 0.0], 
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 100],
    [0, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

p1_camera(params, points)


Matrix R:
1.0, 0.0, 0.0
0.0, 1.0, 0.0
0.0, 0.0, 1.0
Matrix K:
1500.0, 0.0, 2995.0
0.0, 1500.0, 2001.0
0.0, 0.0, 1.0
Matrix Matrix M::
1500.0, 0.0, 2995.0, -29950.0
0.0, 1500.0, 2001.0, -20010.0
0.0, 0.0, 1.0, -10.0
Projections:
0: 10.0 5.0 100.0 => 2084.3 3161.7 inside
1: 0.0 0.0 0.5 => 2001.0 2995.0 inside
2: -30.0 10.0 -20.0 => 1501.0 4495.0 inside
3: 20.0 15.0 20.0 => 4251.0 5995.0 outside
visible:
hidden: 0 1 2 3


In [76]:
'''
Problem 1, Test 2
'''
params = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)


Matrix R:
0.7, -0.1, -0.7
-0.0, 1.0, -0.2
0.7, 0.1, 0.7
Matrix K:
1000.0, 0.0, 3005.0
0.0, 1000.0, 1998.0
0.0, 0.0, 1.0
Matrix M:
-1428.49, -562.45, 2770.03, -5112.73
-1535.59, 617.37, 1500.94, -27388.23
-0.71, -0.18, 0.68, 1.49
Projections:
0: 100.0 15.0 90.0 => 3487.2 -8851.4 outside
1: -100.0 800.0 1500.0 => 3021.6 4043.8 inside
2: 10.0 -500.0 -500.0 => 4311.3 4394.6 outside
3: -30.0 10.0 20.0 => 1589.0 2534.4 inside
visible: 1 3
hidden: 0 2


In [77]:
'''
Problem 1, Test 3
'''
params = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]    
]
p1_camera(params, points)

Matrix R:
0.6, -0.8, 0.2
0.7, 0.7, 0.3
-0.3, -0.0, 0.9
Matrix K:
1000.0, 0.0, 1400.0
0.0, 1000.0, 1000.0
0.0, 0.0, 1.0
Matrix M:
876.13, 1085.63, 1006.88, -59219.44
-580.76, 926.00, 897.35, -19236.55
0.17, 0.27, 0.95, -48.42
Projections:
0: 100.0 15.0 90.0 => 297.8 2323.9 inside
1: -100.0 800.0 1500.0 => 1352.8 1420.4 inside
2: 10.0 -500.0 -500.0 => 1428.5 1672.5 inside
3: -30.0 10.0 20.0 => -794.1 1704.6 outside
visible: 0 1
hidden: 2 3


## Problem 2

In [5]:
def load_points(fn):
    '''
    Input: a path to a file containing x, y points, one point per line.
    Returns: two-d np array where each row contains an x, y point
    '''
    f = open(fn, 'r')
    pts = []
    for line in f:
        line = line.strip().split()
        x, y = float(line[0]), float(line[1])
        pts.append([x, y])
    pts = np.array(pts)
    f.close()
    return pts

In [6]:
def p2_ransac(fn, samples, tau, seed):
    def ransac(points, m, k, tau):

        a = -1 * m
        b = 1
        c = -1 * k

        k_max_new = 0
        in_dist_sum = 0
        out_dist_sum = 0
        for point in points:
            dist = abs(a * point[0] + b * point[1] + c) / math.sqrt(a**2 + b**2)
            if dist <= tau:
                k_max_new += 1
                in_dist_sum += dist
            else:
                out_dist_sum += dist

        return k_max_new, in_dist_sum, out_dist_sum

    def print_line_info(m, k):
        n = math.sqrt(m**2 + 1)
        a = - m / n
        b = 1 / n
        c = - k / n

        if c < 0:
            print("line ({},{},{})".format(round(a, 3), round(b, 3), round(c, 3)))
        else:
            a = m / n
            b = - 1 / n
            c = k / n
            print("line ({},{},{})".format(round(a, 3), round(b, 3), round(c, 3)))


    # MAIN
    if __name__ == '__main__':

        """
        Handle the command-line arguments
        """
        points = load_points(fn)
        np.random.seed(seed)
        sample = []
        N = len(points)
        for i in range(0, samples):
            sample.append(np.random.randint(0, N, 2))

        k_max = 0
        iter = 0
        latest_in_dist_sum = 0
        latest_out_dist_sum = 0
        for index in sample:
            if index[0] == index[1]:
                k_max_new = 0
            else:
                # RANSAC
                # form a line
                i = points[index[0]]
                j = points[index[1]]
                m = (j[1] - i[1]) / (j[0] - i[0])
                k = i[1] - m * i[0]


                # count how many kmax
                (k_max_new, in_dist_sum, out_dist_sum) = ransac(points, m, k, tau)


            # if kmax >, print infos
            if k_max_new > k_max:
                k_max = k_max_new

                print('Sample {}:'.format(iter))
                print('indices ({},{})'.format(index[0], index[1]))
                print_line_info(m, k)
                print('inliers', k_max, '\n')
                latest_in_dist_sum = in_dist_sum
                latest_out_dist_sum = out_dist_sum

            iter += 1
            # at the last iteration of the for loop
            if iter == len(sample):
                in_dist_avg = latest_in_dist_sum / k_max
                out_dist_avg = latest_out_dist_sum / (len(points) - k_max)
                print("avg inlier dist", format(round(in_dist_avg, 3), ".3f"))
                print("avg outlier dist", format(round(out_dist_avg, 3), ".3f"))

In [7]:
'''
Problem 2, Test 1
'''
fn = 'data/p2_pts1_in.txt'
samples = 25
tau = 2.5
seed = 999
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (0,28)
line (-0.983,0.184,-26.286)
inliers 13 

Sample 3:
indices (27,25)
line (0.426,0.905,-4.913)
inliers 19 

Sample 10:
indices (23,4)
line (0.545,0.838,-0.944)
inliers 21 

avg inlier dist 0.739
avg outlier dist 8.920


In [85]:
'''
Problem 2, Test 2
'''
fn = 'data/p2_pts2_in.txt'
samples = 35
tau = 3.0
seed = 1232
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (6,15)
line (-0.023,-1.0,-19.478)
inliers 16 

Sample 2:
indices (46,20)
line (0.178,-0.984,-11.491)
inliers 21 

Sample 4:
indices (75,52)
line (0.5,0.866,-0.018)
inliers 30 

Sample 17:
indices (58,18)
line (-0.408,-0.913,-1.201)
inliers 35 

avg inlier dist 1.383
avg outlier dist 10.267


## Problem 3 (4270 Only)

Students in 6270 should delete Problem 3 cells from their notebooks prior to submission.

In [40]:
'''
Utility for Problem 3
'''
import os

def get_images(img_dir):
    start_cwd = os.getcwd()
    os.chdir(img_dir)
    img_name_list = os.listdir('./')
    img_name_list = [name for name in img_name_list if 'jpg' in name.lower()]
    img_name_list.sort()

    img_list = []
    for i_name in img_name_list:
        im = cv2.imread(i_name, cv2.IMREAD_GRAYSCALE)
        if im is None:
            print('Could not open', i_name)
            sys.exit(0)
        img_list.append(im)

    os.chdir(start_cwd)
    return img_name_list, img_list

In [43]:
def p3_best_focus(image_dir):
    # get the image files from directory
    files = [f for f in os.listdir(image_dir)]
    print (files)

    # sort by file name
    files.sort()

    def getEnergy(f): 
        # obtain the full path of a file
        f = os.path.join(image_dir, f)
        img = cv2.imread(f,cv2.IMREAD_GRAYSCALE)
        im_dx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
        im_dy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
        im_dx_sq = np.square(im_dx)
        im_dy_sq = np.square(im_dy)
        sum_dx = np.sum(im_dx_sq)
        sum_dy = np.sum(im_dy_sq)
        return (sum_dx + sum_dy)/(img.shape[0] * img.shape[1])

    # calculate energy for each image
    max_score = 0
    max_file = ''
    for f in files:
        score = getEnergy(f)
        print ('%s: %.1f' % (f, score))
        if score > max_score:
            max_score = score
            max_file = f
    print ('Image %s is best focused.' % max_file)

In [44]:
image_dir = 'data/evergreen'
p3_best_focus(image_dir)

['DSC_1701.JPG', 'DSC_1700.JPG', 'DSC_1698.JPG', 'DSC_1699.JPG', 'DSC_1697.JPG', 'DSC_1696.JPG']
DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


In [45]:
image_dir = 'data/branches'
p3_best_focus(image_dir)

['DSC_1702.JPG', 'DSC_1703.JPG', 'DSC_1704.JPG', 'DSC_1705.JPG', 'DSC_1707.JPG', 'DSC_1706.JPG']
DSC_1702.JPG: 132.4
DSC_1703.JPG: 1152.0
DSC_1704.JPG: 8229.7
DSC_1705.JPG: 41206.5
DSC_1706.JPG: 22214.3
DSC_1707.JPG: 7876.5
Image DSC_1705.JPG is best focused.
